In [1128]:
from google import genai
from dotenv import load_dotenv
import os
from pydantic import BaseModel, ValidationError
from typing import List
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json
import re
import random as rd

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



*Function to extract Json of LLm output and save it to a file*

In [1129]:
def save_json_from_string(input_string, output_filename):
    """
    Extracts JSON content from a string and saves it to a JSON file with the specified name.

    :param input_string: The input string containing JSON data within brackets.
    :param output_filename: The name of the output JSON file (e.g., "data.json").
    :return: True if the operation was successful, False otherwise.
    """
    # Step 1: Extract the JSON content within the brackets
    result = re.search(r'\[.*\]', input_string, re.DOTALL)

    if result:
        extracted_content = result.group(0)  # Get the matched content

        try:
            # Step 2: Parse the extracted content into a Python object (list/dict)
            json_data = json.loads(extracted_content)

            # Step 3: Write the JSON data to a file
            with open(output_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)  # Save with pretty formatting

            print(f"JSON data has been successfully written to '{output_filename}'.")
            return True  # Operation was successful
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except Exception as e:
            print(f"An error occurred while writing the file: {e}")
    else:
        print("No content found within brackets.")

    return False  # Operation failed

In [1130]:
MODEL = "gemini-2.0-flash-exp"

In [1131]:

COMPANY = 'Proximus'

In [1132]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

**1. SETTING UP A RESEARCH ASSISTANT** (temperature = 0, tools = google search)

In [1133]:
system_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates, metrics, public records and linkedin pages to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write an elaborate company report. 

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report."""

In [1134]:
config = GenerateContentConfig(system_instruction=system_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)

**Main Company report**

In [1135]:
contents = f"""
Write a report about {COMPANY}.

The report should contain an extensive overview of the most important news facts of the last 2 weeks. 

The report should be written in this style example:

Use this JSON schema:

Proximus_news = {{'subject':str, 'overview':str, 'source':str}}
Return: list[Proximus_news]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)


```json
[
    {
        "subject": "Proximus Rebrands International Division as Proximus Global",
        "overview": "Proximus has rebranded its international division, integrating BICS, Telesign, and Route Mobile under a single entity named Proximus Global. This new unit is valued at approximately €3.1 billion in equity and aims to streamline operations, harmonize roles, and drive commercial growth by leveraging the group's global presence and diverse talent. The move is intended to position Proximus as a global leader in digital communications. Clear Bridge Ventures, the investment vehicle of Route Mobile founders, will retain 8.7% of shares in Proximus Global, with Proximus owning the rest.",
        "source": "lightreading.com, telecomtv.com, mobileeurope.co.uk"
    },
    {
        "subject": "Scarlet Offers More Data at Same Price",
        "overview": "Scarlet, a subsidiary of Proximus, has increased the data volumes on its mobile plans at no extra cost, starting in 2025. This 

In [1136]:
save_json_from_string(response.text, 'proximus_news.json')

JSON data has been successfully written to 'proximus_news.json'.


True

**Proximus Key Employees and Roles**

In [1137]:
contents = f"""
Write a report about {COMPANY}

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{COMPANY}.be" (put it all in lower case), the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {{'first_name':str, 'family_name':str, 'role':str, 'department':str, 'email_address':str}}
Return: list[Proximus_employees]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "first_name": "Guillaume",
        "family_name": "Boutin",
        "role": "Chief Executive Officer",
        "department": "CEO's department",
        "email_address": "guillaume.boutin@proximus.be"
    },
    {
        "first_name": "Ben",
        "family_name": "Appel",
        "role": "Corporate Affairs Lead",
        "department": "Corporate Affairs",
        "email_address": "ben.appel@proximus.be"
    },
    {
        "first_name": "Jim",
        "family_name": "Casteele",
         "role": "Consumer Market Lead",
        "department": "Consumer Market",
        "email_address":"jim.casteele@proximus.be"
    },
    {
        "first_name": "Anne-Sophie",
        "family_name": "Lotgering",
        "role": "Enterprise IT Services & Segments Lead",
         "department":"Enterprise Market",
        "email_address":"anne-sophie.lotgering@proximus.be"
    },
    {
        "first_name": "Antonietta",
        "family_name": "Mastroianni",
        "role":"Digital

In [1138]:
save_json_from_string(response.text, 'proximus_employees.json')

JSON data has been successfully written to 'proximus_employees.json'.


True

**Departments Proximus**

In [1139]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Include in the report the full address where the headquarter of the provider is located, the main phone number and vat number.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Proximus_departments = {{'department':str, 'subdivision':list[str], 'address':str, 'phone':str, 'vat':str}}
Return: list[Proximus_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "department": "CEO's Department",
    "subdivision": [],
    "address": "Boulevard du Roi Albert II, 27, 1030 Brussels, Belgium",
    "phone": "0800 22 800",
    "vat": "BE 0202239951"
  },
  {
    "department": "Consumer Market",
    "subdivision": [],
    "address": "Boulevard du Roi Albert II, 27, 1030 Brussels, Belgium",
    "phone": "0800 22 800",
    "vat": "BE 0202239951"
  },
  {
    "department": "Enterprise Market",
    "subdivision": [],
    "address": "Boulevard du Roi Albert II, 27, 1030 Brussels, Belgium",
    "phone": "0800 22 800",
    "vat": "BE 0202239951"
  },
   {
    "department": "Corporate Affairs",
    "subdivision": [],
    "address": "Boulevard du Roi Albert II, 27, 1030 Brussels, Belgium",
    "phone": "0800 22 800",
    "vat": "BE 0202239951"
  },
  {
    "department": "Network & Wholesale",
    "subdivision": [],
    "address": "Boulevard du Roi Albert II, 27, 1030 Brussels, Belgium",
    "phone": "0800 22 800",
    "vat": "BE 0202239951"


In [1140]:
save_json_from_string(response.text, 'proximus_departments.json')

JSON data has been successfully written to 'proximus_departments.json'.


True

**Company Colors**

In [1141]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

Do not include speculative colors and focus only on the main ones.

The report should be written in this style example:

Use this JSON schema:

company_colors = {{'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
Return: list[company_colors]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""


response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "color_name_1": "Primary Purple",
    "hex_code_1": "#5C2D91",
    "rgb_code_1": [92, 45, 145]
  },
  {
    "color_name_2": "White",
    "hex_code_2": "#FFFFFF",
    "rgb_code_2": [255, 255, 255]
  },
   {
    "color_name_3": "Black",
    "hex_code_3": "#000000",
    "rgb_code_3": [0, 0, 0]
  }
]
```


In [1142]:
save_json_from_string(response.text, 'proximus_colors.json')

JSON data has been successfully written to 'proximus_colors.json'.


True

**Blend data from Json files Proximus**

In [1143]:

# File paths for input and output
input_files = {
    "colors": "proximus_colors.json",
    "departments": "proximus_departments.json",
    "employees": "proximus_employees.json",
    "news": "proximus_news.json",
    "providers": "proximus_providers.json"
}
output_file = "blended_data_proximus.json"

# Combine all data into one dictionary
blended_data = {}

for key, file_path in input_files.items():
    with open(file_path, 'r') as f:
        blended_data[key] = json.load(f)

# Write the combined data to a new JSON file
with open(output_file, 'w') as f:
    json.dump(blended_data, f, indent=4)

print(f"Data successfully merged into {output_file}")


Data successfully merged into blended_data_proximus.json


**Service providers**

In [1144]:
contents = f"""

Write a report about {COMPANY}

List the main service providers of the company. Include any known suppliers, contractors, technology service providers, or any other third-party companies that the company relies on. 
Provide the names of the service providers, type and description of the services they provide to the company. Also include the providers homepage in the report, if you can't find it, don not include the provider in the report. Don t include providers with brackets in their company name. 

The report should be written in this style example:

Use this JSON schema:

company_service = {{"provider":str,"service":str,"type":str,"provider_homepage":str]}}
Return: list[company_service]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "provider": "Be-Mobile",
    "service": "Smart mobility solutions",
    "type": "Technology Partner",
    "provider_homepage": "https://www.be-mobile.com/"
  },
  {
    "provider": "ClearMedia",
    "service": "Cloud, cybersecurity and digital workplace solutions",
    "type": "Technology Partner",
    "provider_homepage": "https://www.clearmedia.be/"
  },
   {
    "provider": "Codit",
    "service": "Microsoft Azure cloud solutions",
    "type": "Technology Partner",
    "provider_homepage": "https://www.codit.eu/"
  },
  {
    "provider": "Davinsi Labs",
    "service": "Cybersecurity and operational intelligence",
     "type": "Technology Partner",
    "provider_homepage": "https://www.davinsilabs.com/"
  },
    {
    "provider": "Proximus Spearit",
    "service": "IT solutions",
     "type": "Technology Partner",
    "provider_homepage": "https://www.proximus.com/"
  },
  {
    "provider": "Telindus",
    "service": "ICT services and network integration solutions",

In [1145]:
save_json_from_string(response.text, 'proximus_providers.json')

JSON data has been successfully written to 'proximus_providers.json'.


True

**Providers info** (Proceed with one Provider)

In [1146]:
# Load JSON data from the file
with open("proximus_providers.json", "r") as file:
    providers = json.load(file)

In [1147]:
# Function to pick a random provider
def pick_random_provider(providers):
    if not providers:
        return None  # Handle the case where the list is empty
    return rd.choice(providers)

In [1148]:
provider = pick_random_provider(providers)

print(provider)

{'provider': 'Infosys', 'service': 'IT services', 'type': 'Vendor', 'provider_homepage': 'https://www.infosys.com/'}


In [1149]:
provider = [provider]
file_path = 'provider_general_data.json'

# Write the data to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(provider, json_file, indent=4)

print(f'Data saved to {file_path}')

Data saved to provider_general_data.json


**Provider Key Employees and Roles**

In [1150]:
contents = f"""
    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should only contain a comprehensive summary of the employees in the company.

    The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{provider}.be" (put it all in lower case), the role that they have within the company and the department that they make part of.

    Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found. Make sure the first and the last name contains more then 2 characters. Do not include employees of {COMPANY} in the report.

    Give me as much grounded names that you can find with your research and put them all in the report.

    The report should be written in this style example:

    Use this JSON schema:

    Employees = {{'provider': str, 'first_name': str, 'family_name': str, 'role': str, 'department': str, 'email_address':str}}
    Return: list[Employees]
    """
    
response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)
    

```json
[
    {
        "provider": "Infosys",
        "first_name": "Salil",
        "family_name": "Parekh",
        "role": "Chief Executive Officer and Managing Director",
        "department": "Executive",
        "email_address": "salil.parekh@infosys.be"
    },
    {
        "provider": "Infosys",
        "first_name": "Jayesh",
        "family_name": "Sanghrajka",
        "role": "Chief Financial Officer",
         "department": "Finance",
        "email_address": "jayesh.sanghrajka@infosys.be"
    },
     {
        "provider": "Infosys",
        "first_name": "Shaji",
        "family_name": "Mathew",
        "role": "Chief Human Resources Officer",
         "department": "Human Resources",
        "email_address": "shaji.mathew@infosys.be"
    },
    {
        "provider": "Infosys",
        "first_name": "Inderpreet",
        "family_name": "Sawhney",
        "role": "Chief Legal Officer and Chief Compliance Officer",
        "department": "Legal",
        "email_address": "in

In [1151]:
save_json_from_string(response.text, 'provider_employees.json')

JSON data has been successfully written to 'provider_employees.json'.


True

**Provider Colors**

In [1152]:

contents = f"""

    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

    Do not include speculative colors and focus only on the main ones.

    The report should be written in this style example:

    Use this JSON schema:

    company_colors = {{'provider':str,'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
    Return: list[company_colors]

    Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
    """


response = client.models.generate_content(
        model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "provider": "Infosys",
    "color_name_1": "Blue",
    "hex_code_1": "#007CC3",
    "rgb_code_1": [0, 124, 195]
  }
]
```


In [1153]:
save_json_from_string(response.text, 'provider_color.json')

JSON data has been successfully written to 'provider_color.json'.


True

**Departments Provider**

In [1154]:
contents = f"""

Write a report about company {provider}. This company is a provider of {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Include in the report the full address where the headquarter of the provider is located, the main phone number and vat number.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Provider_departments = {{'provider':str,'department':str, 'subdivision':list[str],'address':str, 'phone':str, 'vat':str}}
Return: list[Provider_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "provider": "Infosys",
    "department": "Infosys Consulting",
    "subdivision": [
      "Digital Experience",
      "Cloud",
      "Data Analytics",
      "Artificial Intelligence",
      "Engineering",
      "Sustainability"
     ],
    "address": "Bengaluru, Plot no. 44/97 A, 3rd cross, Electronic City, Hosur Road, India",
    "phone": "+91 8028520261",
    "vat": "29AAACI4798L2ZT"
  },
  {
    "provider": "Infosys",
    "department": "Infosys Business Process Management",
    "subdivision": [
      "Finance",
      "Procurement",
      "Customer Service",
      "Human Resources"
    ],
     "address": "Bengaluru, Plot no. 44/97 A, 3rd cross, Electronic City, Hosur Road, India",
    "phone": "+91 8028520261",
     "vat": "29AAACI4798L2ZT"
  },
    {
    "provider": "Infosys",
    "department": "EdgeVerve Systems",
    "subdivision": [],
     "address": "Bengaluru, Plot no. 44/97 A, 3rd cross, Electronic City, Hosur Road, India",
    "phone": "+91 8028520261",
    

In [1155]:
save_json_from_string(response.text, 'provider_departments.json')

JSON data has been successfully written to 'provider_departments.json'.


True

**Blend Data from Json files Providers**

In [1156]:
# Function to read JSON data from a file and debug if necessary
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            # Debugging: Print the entire content of the file to inspect the structure
            print(f"Data loaded from {file_path} (Type: {type(data)}): {data}")
            if not isinstance(data, list):
                raise ValueError(f"Data in {file_path} is not a list as expected.")
            return data
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {file_path}")
            return []  # Return an empty list if the JSON is malformed

# Function to blend data
def blend_data_from_files(general_file, employee_file, color_file, department_file):
    # Read data from files
    general_data = read_json_file(general_file)
    employee_data = read_json_file(employee_file)
    color_data = read_json_file(color_file)
    department_data = read_json_file(department_file)
    
    # Merge data
    merged_data = {}

    # Add general data
    for entry in general_data:
        try:
            provider = entry["provider"]
            merged_data[provider] = {"general": entry}
        except KeyError:
            print("KeyError: Missing 'provider' in general data entry.")
            continue

    # Add employee data
    for entry in employee_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "employee" not in merged_data[provider]:
                    merged_data[provider]["employee"] = []
                merged_data[provider]["employee"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in employee data entry.")
            continue

    # Add color data
    for entry in color_data:
        try:
            provider = entry.get("provider", "Unknown")
            if provider in merged_data:
                if "color" not in merged_data[provider]:
                    merged_data[provider]["color"] = {}
                merged_data[provider]["color"].update(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in color data entry.")
            continue

    # Add department data
    for entry in department_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "departments" not in merged_data[provider]:
                    merged_data[provider]["departments"] = []
                merged_data[provider]["departments"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in department data entry.")
            continue

    return merged_data

# File paths
general_file = 'provider_general_data.json'
employee_file = 'provider_employees.json'
color_file = 'provider_color.json'
department_file = 'provider_departments.json'

# Debugging: Print current working directory
print("Current working directory:", os.getcwd())

# Blend the data
blended_result = blend_data_from_files(general_file, employee_file, color_file, department_file)

# Debugging: Print the blended data to verify
print("Blended Data:", json.dumps(blended_result, indent=4))

# Write the result to a new JSON file and capture any potential errors
try:
    with open('blended_data_providers.json', 'w') as output_file:
        json.dump(blended_result, output_file, indent=4)
    print("Blended data has been saved to 'blended_data_providers.json'")
except Exception as e:
    print(f"Error saving file: {e}")






Current working directory: c:\Users\Rik\Desktop\Proximus-Case-Team-4\Rik
Data loaded from provider_general_data.json (Type: <class 'list'>): [{'provider': 'Infosys', 'service': 'IT services', 'type': 'Vendor', 'provider_homepage': 'https://www.infosys.com/'}]
Data loaded from provider_employees.json (Type: <class 'list'>): [{'provider': 'Infosys', 'first_name': 'Salil', 'family_name': 'Parekh', 'role': 'Chief Executive Officer and Managing Director', 'department': 'Executive', 'email_address': 'salil.parekh@infosys.be'}, {'provider': 'Infosys', 'first_name': 'Jayesh', 'family_name': 'Sanghrajka', 'role': 'Chief Financial Officer', 'department': 'Finance', 'email_address': 'jayesh.sanghrajka@infosys.be'}, {'provider': 'Infosys', 'first_name': 'Shaji', 'family_name': 'Mathew', 'role': 'Chief Human Resources Officer', 'department': 'Human Resources', 'email_address': 'shaji.mathew@infosys.be'}, {'provider': 'Infosys', 'first_name': 'Inderpreet', 'family_name': 'Sawhney', 'role': 'Chief Le

**2. SETTING UP THE EMAIL WRITING ASSISTANT** (temperature = 1, tools= ??)

In [1157]:
email_tones = [
    "Urgent",
    "Firm",
    "Action Required",
    "Time-Sensitive",
    "Critical",
    "Pressing",
    "Immediate Response",
    "High-Priority",
    "Polite",
    "Friendly",
    "Professional",
    "Supportive",
    "Respectful",
    "Casual",
    "Empathetic"
]

In [1158]:
random_tone = rd.choice(email_tones)

In [1159]:
print(random_tone)

Polite


In [1160]:
system_instruction= f"""You are a skilled copywriter with a knack for creating emails that feel
    personal, relevant, urgent and engaging. Your task is to write emails to employees of
    {COMPANY} all while maintaining a tone that feels individual, tailored and professional. 
    
    The email should appear to come from a relevant source. Encourage actions like clicking links or downloading attachments with a sense of urgency. Align the email subject with the sender's theme and message.
    
    Incorporate in your answer only complete emails.
    
    Everything about the sender address, subject, and email body is focused on the recipient interacting with the link or attachment.
    
    The email should have a {random_tone}
    """

In [1161]:
config = GenerateContentConfig(system_instruction=system_instruction, temperature=1,maxOutputTokens=8000, top_p=0.9, top_k=40, safety_settings=safety_settings)

**Fetch some Provider Context to generate better emails**

In [1162]:
# Load JSON data from the file
with open("provider_employees.json", "r") as file:
    provider_employees = json.load(file)

In [1163]:
# Load JSON data from the file
with open("provider_departments.json", "r") as file:
    provider_departments = json.load(file)

**Setting up some dummy data about proximus employees**

In [1164]:
proximus_employees = [
    {
        "name": "Marie Declercq", 
        "email": "marie.declercq@proximus.be", 
        "role": "Data Analyst", 
        "language": "Dutch", 
        "department": "Consumer Market"
    },
    {
        "name": "Luc Vandenberg", 
        "email": "luc.vandenberg@proximus.be", 
        "role": "Cybersecurity Specialist", 
        "language": "Dutch", 
        "department": "Network & Wholesale"
    },
    {
        "name": "Emma Wouters", 
        "email": "emma.wouters@proximus.be", 
        "role": "Marketing Specialist", 
        "language": "English", 
        "department": "Enterprise Market"
    },
    {
        "name": "Jai Mehta", 
        "email": "jai.mehta@proximus.be", 
        "role": "System Administrator", 
        "language": "English", 
        "department": "IT Infrastructure"
    },
    {
        "name": "Guillaume Boutin", 
        "email": "guillaume.boutin@proximus.be", 
        "role": "Chief Executive Officer", 
        "language": "French", 
        "department": "Chief Executive Officer's Department"
    }
] 


In [1165]:
random_employee = rd.choice(proximus_employees)

In [1166]:
print(random_employee)

{'name': 'Emma Wouters', 'email': 'emma.wouters@proximus.be', 'role': 'Marketing Specialist', 'language': 'English', 'department': 'Enterprise Market'}


In [1167]:
telecom_employee_events = [
    "Seminars",
    "Presentations",
    "Networking Meetings",
    "Workshops",
    "Training Sessions",
    "Industry Roundtables",
    "Panel Discussions",
    "Webinars",
    "Conferences",
    "Hackathons",
    "Product Demos",
    "Leadership Forums",
    "Strategic Planning Meetings",
    "Customer Experience Sessions",
    "Innovation Labs"
]

In [1168]:
urgent_provider_issues = [
    "Network Outages",
    "SLA Violations",
    "Payment Delays",
    "Contract Breach",
    "Quality Failures",
    "Security Risks",
    "Price Increases",
    "Missed Deadlines",
    "Regulatory Issues",
    "Billing Discrepancies"
]

**Emails Provider to Proximus**

Service related Emails

In [1169]:
contents = f"""

{provider} is a service provider of {COMPANY}. {provider} has these {provider_departments}.

Write some tailored emails based on the role of {random_employee['role']} about an urgent matter that needs to be solved related to {COMPANY} concerning a service that {provider} offers. Elaborate about the issue and point out why it needs to be solved as quick as possible.

The receiver of the email is {random_employee['name']} and should be in {random_employee['language']}. Address the receiver formal and by his last name.

The email signature should contain the first name, family name, role and the company name of one of the employees of {provider}. 

Do not include the link in the body of the email, as it will be implemented separately and placed below the text body, refer to it in the text body where it is placed.

Just plain text in the body, nothing to include anymore.

The email should be written in this style example:

Use this JSON schema:

Provider_email = {{'subject':str,'body':str, 'email_sign_fullname':str, 'email_sign_role':str, 'email_sign_company':str, 'receiver_full_name':str}}
Return: list[Provider_email]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```
[
    {
        'subject': 'Urgent: Proximus Marketing Campaign Data Integration Issue',
        'body': 'Dear Ms. Wouters,\n\nI am writing to you today regarding an urgent issue impacting our current marketing campaign data integration with Proximus. We’ve identified a critical data synchronization problem between our systems, which is now causing inaccurate campaign performance reporting. This directly affects our ability to optimize ongoing initiatives and could lead to misallocation of resources. \n\nImmediate action is required to rectify this and ensure the accurate tracking of campaign metrics. Please find the details and necessary documentation for review and action in the link provided below.\n\nYour prompt attention to this matter is highly appreciated.\n\nSincerely,',
        'email_sign_fullname': 'Priya Sharma',
        'email_sign_role': 'Senior Project Manager',
        'email_sign_company': 'Infosys',
        'receiver_full_name': 'Emma Wouters'
    },
    {
       

repairing a string

In [1170]:
def repair_json_response(response_text):
    """
    Repairs a potentially malformed JSON string from `response.text`.
    Ensures it starts with '[' and ends with ']' after the last complete dictionary.

    Args:
        response_text (str): The response text containing a potentially malformed JSON string.

    Returns:
        None: Prints the repaired JSON string.

    Raises:
        ValueError: If the string cannot be repaired into valid JSON.
    """
    # Remove the unwanted prefix if it exists
    if response_text.strip().startswith("```json"):
        response_text = response_text.replace("```json", "", 1).strip()

    # Ensure the string starts with '['
    if not response_text.strip().startswith("["):
        response_text = "[" + response_text

    # Find the last closing brace ('}')
    last_brace_index = response_text.rfind("}")
    if last_brace_index == -1:
        raise ValueError("No closing brace found in the response text.")

    # Trim the string up to the last complete dictionary and close the list
    repaired_string = response_text[:last_brace_index + 1].rstrip(", \n") + "]"
    
    # Print the repaired JSON string
    print(repaired_string)


In [1171]:
response_llm = response.text
repair_json_response(response_llm)

[```
[
    {
        'subject': 'Urgent: Proximus Marketing Campaign Data Integration Issue',
        'body': 'Dear Ms. Wouters,\n\nI am writing to you today regarding an urgent issue impacting our current marketing campaign data integration with Proximus. We’ve identified a critical data synchronization problem between our systems, which is now causing inaccurate campaign performance reporting. This directly affects our ability to optimize ongoing initiatives and could lead to misallocation of resources. \n\nImmediate action is required to rectify this and ensure the accurate tracking of campaign metrics. Please find the details and necessary documentation for review and action in the link provided below.\n\nYour prompt attention to this matter is highly appreciated.\n\nSincerely,',
        'email_sign_fullname': 'Priya Sharma',
        'email_sign_role': 'Senior Project Manager',
        'email_sign_company': 'Infosys',
        'receiver_full_name': 'Emma Wouters'
    },
    {
      

In [1172]:
save_json_from_string(response_llm, 'email_provider_to_proximus.json')

Error parsing JSON: Expecting property name enclosed in double quotes: line 3 column 9 (char 16)


False

**Internal Emails Proximus**

**4. BLENDING DATA IN HTML**

**5. SENDING MAILS TO GOPHISH**